# Experience

## Import module

In [ ]:
from utils import BasicDataset
from unet import UNet
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.scenarios.dataset_scenario import benchmark_from_datasets
from avalanche.evaluation.metrics import WeightCheckpoint, gpu_usage_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, disk_usage_metrics
from avalanche.benchmarks.utils import AvalancheDataset
from avalanche.logging import InteractiveLogger, TextLogger, WandBLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.supervised import Naive,EWC
from avalanche.benchmarks.scenarios.supervised import class_incremental_benchmark
from torch import optim
import numpy as np
import wandb

## Prepare Dataset

In [ ]:
from avalanche.benchmarks import benchmark_from_datasets
from avalanche.benchmarks.utils.data_attribute import DataAttribute
import torch

In [ ]:
aval_datasets = []
for i in range(5):
    train_dataset = BasicDataset(images_dir=f"./stack_data/train/img/{i}",
                                mask_dir=f"./stack_data/train/label/{i}")
    aval_dataset = AvalancheDataset([train_dataset],
                                    data_attributes=[
                                                    DataAttribute([0] * len(train_dataset), "targets_task_labels"),
                                                    ])
    aval_datasets.append(aval_dataset)

In [ ]:
bm = benchmark_from_datasets(train=aval_datasets,
                             test=[aval_datasets[0]]*len(aval_datasets))

In [ ]:
print(f"original training samples = {len(bm.train_stream[0].dataset)}")

## Settings

In [ ]:
epochs = 1
batch_size= 8
learning_rate = 0.00001
weight_decay = 0.00000001
momentum = 0.999

## Train using NAIVE

In [ ]:
func_name = "NAIVE_all"

In [ ]:
model = UNet(n_channels=3, n_classes=4)

In [ ]:

# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger(f"./log/tb_data/{func_name}")

wandb_logger = WandBLogger(project_name="UNet_CL",
                           run_name=func_name,
                           path=f"./log/checkpoint/{func_name}",
                           dir=f"./log/wandb/{func_name}")

# log to text file
text_logger = TextLogger(open(f'./log/{func_name}.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

In [ ]:
# CREATE THE STRATEGY INSTANCE (NAIVE)
optimizer = optim.RMSprop(model.parameters(),
                          lr=learning_rate, 
                          weight_decay=weight_decay, 
                          momentum=momentum, 
                          foreach=True)

eval_plugin = EvaluationPlugin(
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(experience=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    gpu_usage_metrics(gpu_id=0, minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger, wandb_logger]
)

In [ ]:
from avalanche.training import Naive
cl_strategy = Naive(
    model=model, 
    optimizer=optimizer,
    criterion=CrossEntropyLoss(), 
    train_mb_size=batch_size, 
    train_epochs=epochs,
    device="cuda:0",
    eval_mb_size=batch_size,
    evaluator=eval_plugin)



In [ ]:
import torch
import torch.nn as nn
import torch.nn.init as init

# 在某个时刻，你想重新初始化模型的参数
def reinitialize_model(model, initialization_method="normal", mean=0, std=0.02):
    for name, param in model.named_parameters():
        if "weight" in name:
            if initialization_method == "normal":
                init.normal_(param, mean=mean, std=std)
            elif initialization_method == "uniform":
                init.uniform_(param, a=-std, b=std)  # 注意调整a和b以匹配你的需求
            elif initialization_method == "xavier":
                init.xavier_normal_(param)
            elif initialization_method == "kaiming":
                init.kaiming_normal_(param, nonlinearity='relu')
            else:
                raise ValueError(f"Unsupported initialization method: {initialization_method}")
        elif "bias" in name:
            init.zeros_(param)  # 通常将偏置初始化为0

# 使用指定的方法重新初始化模型参数
reinitialize_model(model, initialization_method="normal")  # 使用正态分布重新初始化权重

# 如果需要，可以多次调用此函数以应用不同的初始化策略

In [ ]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for i, experience in enumerate(bm.train_stream):
# if True:
    # print("Start of experience: ", experience.current_experience)
    # print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(bm.test_stream[i]))
    reinitialize_model(cl_strategy.model, initialization_method="normal") 

In [ ]:
import pickle
pickle.dump(results,open(f"./log/eval/{func_name}.pkl","wb"))

In [ ]:
wandb.finish()